In [3]:
import os

def get_os_env_var(var_name):
    return os.environ[var_name]

def check_version():
    # Check python and Azure CLI version

    !python --version
    !az --version

    # Check Azure ML SDK installed
    import azureml.core
    from azureml.core import Workspace

    print("SDK version:", azureml.core.VERSION)
    
# Enter the resource group in Azure where you want to provision the resources 
resource_group_name = get_os_env_var("resource_group_name")

# Enter Azure region where your services will be provisioned, for example "eastus2"
azure_region=get_os_env_var("azure_region")

# Provide your Azure subscription ID to provision your services
subscription_id =get_os_env_var("subscription_id")

# Provide your Azure ML service workspace name 
# If you don't have a workspace, pick a name to create a new one
aml_workspace_name = get_os_env_var("aml_workspace_name")

In [4]:
# Azure Login - should open a browser page to login to Azure
!az login

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
Unable to revert mtime: /usr/local/share/fonts
Opening in existing browser session.
You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureCloud",
    "id": "2b76f630-69ca-40e9-a1ce-69d0d46fcca8",
    "isDefault": false,
    "name": "MineStar EXT QA",
    "state": "Enabled",
    "tenantId": "ceb177bf-013b-49ab-8a9c-4abce32afc1e",
    "user": {
      "name": "Hung_Alice_P@cat.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "2643b3a6-256f-49a8-81a2-16283d061433",
    "isDefault": true,
    "name": "MineStar EXT Dev",
    "state": "Enabled",
    "tenantId": "ceb177bf-013b-49ab-8a9c-4abce32afc1e",
    "user": {
      "name": "Hung_Alice_P@cat.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "2dc23ad0-abb9-4908-a669-196ef7956029",
    "isDefault

In [1]:
from azureml.core.model import Model
from azureml.core import Workspace

def load_workspace():
    import os.path
    if os.path.isfile("aml_config/config.json"):
        ws = Workspace.from_config()
    else:
        ws = Workspace.create(subscription_id = subscription_id,
                    resource_group = resource_group_name,
                    name = aml_workspace_name,
                    location = azure_region)
        ws.write_config() # Write config file to aml_config/config.json
    return ws
    print("Using Azure ML Workspace '" + ws.name + "'")
    #print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

def register_model(ws, model_path, model_name, tags=None, description=None):
    model = Model.register(model_path = model_path,
                           model_name = model_name,
                           tags = tags,
                           description = description,
                           workspace = ws)
    return model

def get_model(ws, model_name):
#     model_name = "azureml-anormaly-model"
#     model_path="model.pkl"
    return Model(workspace=ws, name=model_name)
    
def post_login():
    # Set subscription after login
    print('Setting subscription...')
    !az account set --subscription $subscription_id